In [ ]:
# import requests

# # replace with your VM's IP address
# url = 'http://127.0.0.1:8080/transcribe'  

# # Here we are using local file Audio123.wav to send as json 
# # POST request
# file = {'file': open('Audio123.wav', 'rb')}
# response = requests.post(url, files=file)

# if response.ok:
#     transcription = response.json()['transcription']
#     print(transcription)
# else:
#     print('Error:', response.reason)

In [ ]:
# pip install gradio flask transformers waitress scipy

In [1]:
import gradio as gr
from transformers import pipeline
from gradio.components import Audio, Textbox
from scipy.io.wavfile import write
import numpy as np
import torchaudio
# # specify the device to use
device = 0  # set to -1 for CPU, 0 for first GPU, 1 for second GPU, etc.
device_str = f"cuda:{device}" if device >= 0 else "cpu"
# device_str ="cpu"

# load the pipeline with the specified device
pipe = pipeline("automatic-speech-recognition", model="bofenghuang/whisper-large-v2-french", device=device_str)

# set the forced decoder IDs
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language="fr", task="transcribe")        

def transcribe_audio(audio):
    sample_rate, audio_tensor  = audio
    
    write("example.wav", sample_rate, audio_tensor.astype(np.int16))
    
    waveform, sample_rate = torchaudio.load('example.wav')
    
    audio_tensor = waveform
    
    required_sample_rate = 16000  
    if sample_rate != required_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=required_sample_rate)
        audio_tensor = resampler(audio_tensor)
    
    
    audio_tensor = audio_tensor.squeeze().numpy()
    generated_sentences = pipe(audio_tensor, max_new_tokens=1000)["text"]
    
    return generated_sentences

audio_input = Audio(source="upload", label="Upload Audio File")
text_output = Textbox(label='Transcription')

interface = gr.Interface(fn=transcribe_audio, inputs=audio_input, outputs=text_output, 
                         title="Whisper Audio Transcription", 
                         description="Transcribe audio to text using the Facebook Whisper model.",
                         )
interface.queue().launch(server_name='0.0.0.0', server_port=8081)

Running on local URL:  http://0.0.0.0:8081

To create a public link, set `share=True` in `launch()`.
